In [11]:
import pandas as pd
import pickle
import numpy as np
import os

In [12]:
os.getcwd()

'c:\\Users\\Multimatics\\Documents\\halalina-ml\\sourceCode'

In [13]:
# Load model terbaik (pipeline lengkap) dari file
with open('c:/Users/Multimatics/Documents/halalina-ml/model/best_random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Data baru sebagai DataFrame (pastikan kolom sama seperti X saat training)
new_data = pd.DataFrame([{
    'job': 'white-collar',
    'marital': 'married',
    'balance': 15000000000,
    'age_group': 'gen_x',
    'is_having_debt': 2
}])

# Lakukan prediksi
prediction = loaded_model.predict(new_data)

# Jika ingin probabilitas per kelas
proba = loaded_model.predict_proba(new_data)

print(f"🎯 Prediksi Klaster untuk Data Baru: {prediction}")
print(f"🎯 Confidence Score masing - masing Klaster untuk Data Baru: {proba}")

🎯 Prediksi Klaster untuk Data Baru: [2]
🎯 Confidence Score masing - masing Klaster untuk Data Baru: [[0.01332632 0.02237578 0.9642979 ]]


In [ ]:
# Mapping klaster ke profil risiko
risk_profiles = {
    0: "Agresif",
    1: "Moderat",
    2: "Defensif"
}

# Base model alokasi portofolio (persentase) untuk masing-masing profil
base_allocations = { 
    "Agresif": {
        "Tabungan Emas": 0.40,
        "SBSN": 0.30,
        "RDPU Syariah": 0.20,
        "Deposito Syariah": 0.10
    },
    "Moderat": {
        "Tabungan Emas": 0.25,
        "SBSN": 0.35,
        "RDPU Syariah": 0.25,
        "Deposito Syariah": 0.15
    },
    "Defensif": {
        "Tabungan Emas": 0.05,
        "SBSN": 0.40,
        "RDPU Syariah": 0.30,
        "Deposito Syariah": 0.25
    }
}

# Ambil probabilitas per kelas dari model
proba_scores = np.array(proba).flatten()

# Hitung total alokasi berbobot dari semua base model dikalikan proba
final_allocation = {
    "Tabungan Emas": 0.0,
    "SBSN": 0.0,
    "RDPU Syariah": 0.0,
    "Deposito Syariah": 0.0
}

# Bobot: proba[0] = agresif, proba[1] = moderat, proba[2] = defensif
for i, profile in enumerate(["Agresif", "Moderat", "Defensif"]):
    for product in final_allocation:
        final_allocation[product] += base_allocations[profile][product] * proba_scores[i]

# Konversi ke persen (float)
percent_values = {k: v * 100 for k, v in final_allocation.items()}

# Simpan pembulatan ke bawah dan sisa desimal
floored = {k: int(v) for k, v in percent_values.items()}
remainders = {k: percent_values[k] - floored[k] for k in percent_values}

# Hitung total awal dan selisih
total_floored = sum(floored.values())
diff = 100 - total_floored

# Urutkan berdasarkan sisa desimal terbesar → tambahkan 1 ke elemen tersebut hingga total = 100
sorted_remainders = sorted(remainders.items(), key=lambda x: x[1], reverse=True)

# Tambahkan 1 ke elemen yang sisa desimalnya terbesar
for i in range(diff):
    floored[sorted_remainders[i][0]] += 1

# Tampilkan hasil
predicted_cluster = int(prediction[0])
risk_profile = risk_profiles[predicted_cluster]

print(f"🎯 Prediksi Klaster: {predicted_cluster}")
print(f"🧠 Profil Risiko Dominan: {risk_profile}")
print(f"📈 Confidence Score: {proba_scores}")
print("📊 Rekomendasi Portofolio Berdasarkan Confidence Score (Total 100%):")

for product, percent in floored.items():
    print(f"- {product}: {percent}%")

🎯 Prediksi Klaster: 2
🧠 Profil Risiko Dominan: Defensif
📈 Confidence Score: [0.01332632 0.02237578 0.9642979 ]
📊 Rekomendasi Portofolio Berdasarkan Confidence Score (Total 100%):
- Tabungan Emas: 6%
- SBSN: 40%
- RDPU Syariah: 30%
- Deposito Syariah: 24%
